# A Minimal Specviz+notebook Workflow

This notebook provides a short example of combining the Specviz interactive visualization tool of the `jdaviz` package with a more traditional non-interactive Python workflow.  The science case is loading a single 1D-spectrum (from the [Sloan Digital Sky Survey](https://www.sdss.org/)) and measuring the flux in a single spectral line (${\rm H}\alpha$).

We begin by creating an instance of the `Specviz` helper class, which provides a range of conveniences for the discerning astronomy to easily work with the visualization tool.  Ending the cell with the `.app` attribute of that instance will show the viz tool.

In [ ]:
from jdaviz import Specviz

specviz = Specviz()
specviz.show()

The above is currently empty. While one could use the "import" option to find a local file on disk, a notebook workflow is more amenable to downloading and loading a spectrum directly in Python code.  To do this, we load our spectrum using the `specutils` package. This provides maximum flexibility because `Spectrum1D` objects can either be created from local data files, URLs (as shown below), or manually from user-provided arrays.

We then use the `Specviz.load_data` method to load the data into the array - this should then immediately show the spectrum in the cell above. 

In [ ]:
import specutils

spec_url = 'https://dr14.sdss.org/optical/spectrum/view/data/format=fits/spec=lite?plateid=1323&mjd=52797&fiberid=12'
spec = specutils.Spectrum1D.read(spec_url, cache=True)

specviz.load_spectrum(spec)

That spectrum looks great! But the line we are looking for is pretty narrow.  We could use the UI to zoom, which can be done using the pan/zoom tool, but you can also execute the cell below to zoom the view in on the region around ${\rm H}\alpha$:

In [ ]:
# zoom in on Halpha region
v = specviz.app.get_viewer('spectrum-viewer')
v.state.x_min = 6500
v.state.x_max = 6750

If the spectrum has uncertainties, we can display them as a shadded band around the spectral trace.

In [ ]:
v.show_uncertainties()

If the spectrum has masked data points, we can mark them on the plot.

In [ ]:
v.show_mask()

This erases the uncertainties and masks from the plot.

In [ ]:
v.clean()

Now use the Glupyter range selection tool (expand the menu and choose the second tool), and select the area around the ${\rm H}\alpha$ line.  Then you can execute the cell below to get that selection into a format `specutils` understands:

In [ ]:
line_region = specviz.get_spectral_regions()['Subset 1']
line_region

In [ ]:
# To reproduce the exact values this notebook was written assuming, uncomment the below
# line_region = specutils.SpectralRegion(6557.48830955*u.angstrom, 6584.69919391*u.angstrom)

Now with that region selected, we can build a Gaussian + Constant continuum model to fit the selected line, and then fit it to just the data in the selected region:

In [ ]:
from astropy.modeling import models
from specutils.fitting import fit_lines
from  specutils import manipulation

line_model_guess = models.Gaussian1D(mean=(line_region.lower + line_region.upper)/2, 
                                     stddev=3, 
                                     amplitude=1000) + models.Const1D(200)

#fit that model to the selected region

# after a bug fix, the below should just be a single line:
# fit_lines(spec, line_model_guess, window=line_region)

extracted = manipulation.extract_region(spec, line_region)
extracted.mask[:] = False
fitted_line = fit_lines(extracted, line_model_guess)

fitted_line

Now we plot that model with the spectrum to examine the fit:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from astropy import units as u

plt.plot(spec.spectral_axis, spec.flux, lw=3)

model_lamb = np.linspace(v.state.x_min, v.state.x_max, 1000)*u.angstrom
plt.plot(model_lamb, fitted_line(model_lamb), '-', lw=2)

plt.xlim(v.state.x_min, v.state.x_max)
plt.ylim(v.state.y_min, v.state.y_max);

Looks good! 

Now to achieve the final goal of a line flux measurement, we can integrate over the line: 

In [ ]:
from scipy.integrate import quad

quad(fitted_line.unitless_model.left, 6500, 6700)[0] * fitted_line.return_units*spec.spectral_axis.unit